# Part 2: Elasticsearch and Generating Reports

In [1]:
import os
from dotenv import load_dotenv 

from elasticsearch import Elasticsearch, helpers

from elasticsearch.exceptions import RequestError

import pandas as pd
import unicodedata

import nltk
from nltk.corpus import stopwords

#### Conect to Elasticsearch API
- Create and import environment variables `ELASTIC_SEARCH_API_ENDPOINT` and `ELASTIC_SEARCH_API_KEY`
- Creates Index if it doesn't already exist
- Note: Settings are set to `my_ascii_folding_filter` which means elastic search will treat accented characters as it's non-accented verision and itself

In [2]:

index = "courses"
load_dotenv() 
client = Elasticsearch(
  os.getenv("ELASTIC_SEARCH_API_ENDPOINT"),
  api_key=os.getenv("ELASTIC_SEARCH_API_KEY"),
  # verify_certs=False
)
try:
  print("Client Info:", client.info())
except ElasticsearchException as e:
  print(f"Error connecting to Elasticsearch: {e}")

settings = {
    "analysis": {
        "filter": {
            "my_ascii_folding_filter": {
                "type": "asciifolding",
                "preserve_original": True
            }
        },
        "analyzer": {
            "standard_asciifolding": {
                "tokenizer": "standard",
                "filter": ["lowercase", "my_ascii_folding_filter"]
            }
        }
    }
}

mappings = {
    "properties": {
        "course": {"type": "text", "analyzer": "standard_asciifolding"},
        "CRN": {"type": "text", "analyzer": "standard_asciifolding"},
        "subjectDescription": {"type": "text", "analyzer": "standard_asciifolding"},
        "courseTitle": {"type": "text", "analyzer": "standard_asciifolding"},
        "description": {"type": "text", "analyzer": "standard_asciifolding"},
    }
}

try:
    client.indices.create(index=index, mappings=mappings, settings=settings)
    print(f"Index '{index}' created successfully.")
except RequestError:
    print(f"Index '{index}' already exists.")

Client Info: {'name': 'instance-0000000000', 'cluster_name': 'a6d98f7b24ab431881958cb334894f4f', 'cluster_uuid': 'ToWvQ6beRcCo4O74i9TPtA', 'version': {'number': '8.14.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2afe7caceec8a26ff53817e5ed88235e90592a1b', 'build_date': '2024-07-01T22:06:58.515911606Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Index 'courses' already exists.


#### Bulk Load Data into Index

In [4]:

df_courses = pd.read_csv("courses.csv")
def generate_docs():
    for _, row in df_courses.iterrows():
        yield {
            "_index": index,
            "_id":row["courseNumber"],
            "CRN": row["courseReferenceNumber"],
            "courseTitle": row["courseTitle"],
            "subjectDescription": row["subjectDescription"],
            "description": row["description"],
        }

helpers.bulk(client, generate_docs())

(1685, [])

#### Preprocess Keywords/Phrases
Reasoning behind data cleaning decisions
- Query search default operator is `AND` because `OR` leads to many redundancies 
- `{}` were replaced with `()` because elasticsearch query search doesn't allow `{}`

* `;` were replaced with ` OR ` and `()` wraping the phrases between each `;` in because there are multiple phrases in 1 entry
  - For Example: `"Poverty Alleviation Programs; Poverty Alleviation Programme; Poverty Alleviation Programmes"` is cleaned to `"(Poverty Alleviation Programs) OR (Poverty Alleviation Programme) OR (Poverty Alleviation Programmes)"`
- `" / "` is converted to `OR` and `()` wraping the phrases between each `" / "` because there are multiple phrases in 1 entry
  - For Example: `"Doha Development Round / Doha Round"` is converted to `"(Doha Development Round) OR (Doha Round)"`
- `"/"` is converted to `OR` and no `()` wraping the phrases between each `"/"` because `"/"` separtes two words instead of two phrases like with `" / "`
  - For Example: `"Gender equality/parity"` is converted to `"(Gender equality OR parity)"`
  - Note: 1 inconsistent entry in USC keywords list is `"Sustainable building/s"` and this is converted to `"Sustainable building OR s`
  - Extra Possible Solution: Replace `/` with wildcard `*` in order to allow for more possible words, however search time may increase
- `"-"` is converted to `OR` and no `()` wraping the words
  - For Example: `"Micro-organisms"` is converted to `"Micro OR organisms"`
  - Note: Change to `AND` if for both parts of hyphenated word need to be in course 
  - Extra Possible Solution: Replace `-` with wildcard `*` in order to allow for more possible words, however search time may increase
* All strings are unicode normalized to `NFKC` standard in order to replace non-breaking spaces with regular spaces
  - For Example: `\xa0` is a non-breaking space and it is converted to `" "` 
* All stop words are removed using the `NLTK` library because common words such as lowercase `"and, "or", "not", be, being, is, the, etc...` should not be requred to be in search query
  - Note: 1 entry in USC keywords list is `"wellbeing/well being/well-being"` and the stop words library did remove `"being`"

In [5]:
df_UoA = pd.read_csv('./UoA-SDG-Keyword-List-Ver.-1.1.xlsx - SDG Keywords (initial ver.).csv')
df_UoA = df_UoA[["SDG Keywords", "Alternatives"]]
df_UoA = pd.concat([df_UoA['SDG Keywords'], df_UoA['Alternatives']], ignore_index=True)
df_UoA = df_UoA.to_frame(name='Keywords')
df_UoA.dropna(inplace=True, ignore_index=True)

df_USC = pd.read_csv('./USC-Compiled-Keywords-for-SDG-Mapping_Final_17-05-10.xlsx - Compiled SDG Keywords.csv')
df_USC = pd.concat([df_USC[col] for col in df_USC.columns], ignore_index=True)
df_USC = df_USC.to_frame(name='Keywords')
df_USC.dropna(inplace=True, ignore_index=True)

combined_df = pd.concat([df_UoA, df_USC], ignore_index=True)
combined_df.dropna(inplace=True, ignore_index=True)
keywords_list = combined_df['Keywords'].tolist()
keywords = set(keywords_list)
keywords = sorted(list(keywords))

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

removed_stop_words = set()

for i, phrase in enumerate(keywords):
    new_str = phrase

    if "{" in phrase:
        new_str = new_str.replace("{", "(")
    if "}" in phrase:
        new_str = new_str.replace("}", ")")

    if ";" in phrase:
        segments = new_str.split(";")
        segments = [f"({segment.strip()})" for segment in segments]
        new_str = " OR ".join(segments)
        
    if " / " in phrase:
        segments = new_str.split(" / ")
        segments = [f"({segment.strip()})" for segment in segments]
        new_str = " OR ".join(segments)
        
    if "/" in phrase:
        segments = new_str.split("/")
        segments = [f"{segment.strip()}" for segment in segments]
        new_str = " OR ".join(segments)
    
    if "-" in phrase:
        segments = new_str.split("-")
        segments = [f"{segment.strip()}" for segment in segments]
        new_str = " OR ".join(segments) 

    #normalize the data
    new_str = unicodedata.normalize('NFKC', new_str)

    #remove stop words except for AND, OR, and NOT
    words = new_str.split(" ")
    filtered_words = [
        word for word in words 
        if word.lower() not in stop_words or word.isupper()
    ]
    removed_stop_words.update([
        word for word in words 
        if word.lower() in stop_words and not word.isupper()
    ])
    new_str = ' '.join(filtered_words)
    keywords[i] = new_str
print("Removed Words:", removed_stop_words)



len(keywords)

[nltk_data] Downloading package punkt to /home/safeduck/nltk_data...


Removed Words: {'Over', 'Not', 'the', 'in', 'No', 'of', 'In', 'against', 'To', 'being', 'All', 'and', 'The', 's', 'or', 'no', 'to', 'And', 'with', 'Being', 'all', 'for', 'Under', 'For', 'on', 'Against'}


[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/safeduck/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


3465

#### Write Keywords to CSV File

In [6]:
import csv
with open('keywords.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in keywords:
        writer.writerow([item])

#### Create Report
- The file `report.txt` contains the all keywords with matching courses

In [11]:
total_matches = 0
courses_matched = set()
with open('report.txt', 'w') as f:
    for phrase in keywords:
        result = client.search(index="courses", query={"query_string": {"query": phrase.lower(), "default_field": "description","analyze_wildcard": True, "default_operator": "AND"}}, size=10000)
        total_matches += result["hits"]["total"]["value"]
        
        f.write(f"-------- {phrase} --------\n")
        f.write(f"Total Class Count: {len(result['hits']['hits'])}\n")

        for hit in result["hits"]["hits"]:
            courseNumber = hit["_id"]
            courses_matched.add(courseNumber)
            courseTitle = hit["_source"]["courseTitle"]
            subjectDescription = hit["_source"]["subjectDescription"]
            f.write(f"{subjectDescription}: {courseNumber} - {courseTitle}\n")

        f.write("\n")

    f.write(f"Total matches: {total_matches}\n")
    f.write(f"Total courses: {len(courses_matched)}\n")